In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import scipy.io as scio
import numpy as np
import matplotlib.pyplot as plt

tf.__version__ 

'2.4.1'

In [2]:
Ne = 128 # number of transducer elements

# 1. Reproduce ABLE

## 1-1. Build a model

In [3]:
def antirectifier(x):   #copy from https://www.tensorflow.org/.../keras/layers/Lambda
    x -= K.mean(x, axis=1, keepdims=True)
    x = K.l2_normalize(x, axis=1)
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=1)

In [4]:
_input = keras.Input(shape=(Ne,))
x = keras.layers.Dense(Ne)(_input)
x = keras.layers.Lambda(antirectifier)(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(Ne/4)(x)
x = keras.layers.Lambda(antirectifier)(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(Ne/4)(x)
x = keras.layers.Lambda(antirectifier)(x)
x = keras.layers.Dropout(0.2)(x)

weights = keras.layers.Dense(Ne, name = "weights")(x)  # no activation is applied
sumWeights = keras.layers.Lambda(lambda x: K.sum(x, axis=1, keepdims=True), name = "sumWeights")(weights) 

multiplied = keras.layers.Multiply()([_input, weights])
output = keras.layers.Lambda(lambda x: K.sum(x, axis=1, keepdims=True), name="output")(multiplied)

model = keras.Model(
    inputs=[_input],
    outputs=[output,sumWeights],
)

#uncomment to see the architecture of the model
#keras.utils.plot_model(model, "ABLE_model_with_shape_info.png", show_shapes=True)  

## 1-2 Define loss functions and compile

In [5]:
def loss_SMSLE(y_true, y_pred, scale=1.):
    first_log = K.log(K.clip(K.abs(y_pred[:, :1]) * scale, K.epsilon(), None) + 1.) * K.sign(y_pred[:, :1])
    second_log = K.log(K.clip(K.abs(y_true[:, :1]) * scale, K.epsilon(), None) + 1.) * K.sign(y_true[:, :1])
    return K.mean(K.square(first_log - second_log), axis=-1)

In [6]:
def loss_unity(y_true, y_pred):
    return K.mean( K.square(y_pred - y_true),  axis=0 )

In [7]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=1e-3),
    loss={"output":loss_SMSLE, "sumWeights":loss_unity},
    loss_weights=[1,1]
)

## 1-3. Prepare x_train / y_train / y_train_weights

In [8]:
inputFilepath = r'E:\Desktop\internshipDeepUS\04_16_2021_(ABLE)\Input_Target_ABLE\input\PICMUS16.mat'
targetFilepath = r'E:\Desktop\internshipDeepUS\04_16_2021_(ABLE)\Input_Target_ABLE\target\PICMUS16.mat'
datasetsNames = ['carotid_cross','carotid_long','contrast_expe','resolution_expe','contrast_simu','resolution_simu']

In [9]:
inputFile = scio.loadmat(inputFilepath)
targetFile = scio.loadmat(targetFilepath)

In [10]:
print("input shape of each dataset:",inputFile[datasetsNames[1]].shape)
print("target shape of each dataset:",targetFile[datasetsNames[1]].shape)

input shape of each dataset: (235683, 128)
target shape of each dataset: (235683, 1)


In [11]:
x_train = np.concatenate([inputFile[datasetsNames[i]] for i in range(len(datasetsNames))],axis=0) # x_train.shape=(1414098, 128)
y_train = np.concatenate([targetFile[datasetsNames[i]] for i in range(len(datasetsNames))],axis=0)# y_train.shape=(1414098, 1)

In [12]:
y_train_weights = np.ones([1414098,1]) # y_train_weights.shape=(1414098, 1)

## 1-3. Train and save the model

In [13]:
val_split = 0.3
EPOCHS = 10
checkpoint_filepath = '../tmp/checkpoint'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

early_stop_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3)

In [14]:
# In this case, The model.compile()'s argument "loss_weights=[1,1]", which is equal to lambda = 0.5 
# (lambda is the weight ratio  who is defined in ABLE paper)

# Model weights are saved at the end of every epoch, if it's the best seen so far.
model.fit(x=x_train, y=[y_train, y_train_weights], 
          validation_split=val_split, epochs=EPOCHS, 
          callbacks=[model_checkpoint_callback,early_stop_callback])


Epoch 1/10
30934/30934 [==============================] - 201s 6ms/step - loss: 1.7578 - output_loss: 1.7523 - sumWeights_loss: 0.0055 - val_loss: 1.5884 - val_output_loss: 1.5884 - val_sumWeights_loss: 3.2311e-05
Epoch 2/10
30934/30934 [==============================] - 193s 6ms/step - loss: 0.9766 - output_loss: 0.9738 - sumWeights_loss: 0.0028 - val_loss: 1.6664 - val_output_loss: 1.6652 - val_sumWeights_loss: 0.0012
Epoch 3/10
30934/30934 [==============================] - 186s 6ms/step - loss: 0.8266 - output_loss: 0.8237 - sumWeights_loss: 0.0029 - val_loss: 1.7267 - val_output_loss: 1.7264 - val_sumWeights_loss: 2.4857e-04
Epoch 4/10
30934/30934 [==============================] - 188s 6ms/step - loss: 0.7517 - output_loss: 0.7487 - sumWeights_loss: 0.0030 - val_loss: 1.7866 - val_output_loss: 1.7733 - val_sumWeights_loss: 0.0133


In [ ]:
# model.save("my_model.tf")
# model.save_weights("my_weights.tf")

# If set "loss_weights=[1,50]"

# model.fit(
#     x=x_train, y=[y_train, y_train_weights], batch_size=None, epochs=4, verbose=1
# )

# Epoch 1/4
# 44191/44191 [==============================] - 247s 5ms/step - loss: 2.5075 - output_loss: 2.3395 - sumWeights_loss: 0.0034
# Epoch 2/4
# 44191/44191 [==============================] - 245s 6ms/step - loss: 2.2096 - output_loss: 2.0750 - sumWeights_loss: 0.0027
# Epoch 3/4
# 44191/44191 [==============================] - 227s 5ms/step - loss: 2.0816 - output_loss: 1.9278 - sumWeights_loss: 0.0031
# Epoch 4/4
# 44191/44191 [==============================] - 229s 5ms/step - loss: 1.9868 - output_loss: 1.8230 - sumWeights_loss: 0.0033